In [1]:
# %pip install torch torchvision torchaudio
# to install pytorch on jupyter

In [1]:
import numpy as np
import torch

# Phase1

- 𝑡∈[0,10] , t is a grid of time points which defeines the solution space of the differential equation 
- 

In [27]:
t_min, t_max= 0.0, 10.0
points= 200
torch.manual_seed(0)
t_grid= np.linspace(t_min, t_max,points)
t= torch.tensor(t_grid, dtype=torch.float32).view(-1,1) # shape is 200,1

#### defining initial conditions

In [37]:
# runu this on your pc, you need this
x0, v0 = 1.0, 0.0

t_ic= torch.tensor([[0.0]], dtype=torch.float32)    # time= 0
x_ic= torch.tensor([[x0]], dtype=torch.float32)     # displacement at t=0 is x=1
v_ic= torch.tensor([[v0]], dtype=torch.float32)

#### Collocation Points (Physics Loss)
- Collocation points are artificially chosen points inside the problem’s domain (here, the time interval[0,10]) where the neural network is forced to satisfy the given differential equation.
- Instead of training only on data (like in normal ML), we also enforce the Ordinary Differential Equation(ODE) itself
- To do this, we sample collocation points inside the time interval  𝑡∈[0,10]
- At each collocation point, we ask the network: Does predicte x(t) satisfy the differential equation

In [ ]:
# do not run this cell again. go to next cell
n_collocationPoints= 100
torch.manual_seed(0)
t_collocation= torch.rand(n_collocationPoints, 1) * (t_max-t_min) + t_min

In [36]:
# do not run this
#torch.save(t_collocation, "collocation.pt")

# PHASE 2

In [ ]:
# to use the collocation points, do t_collocation= torch.load("collocation.pt")
t_collocation= torch.load("collocation.pt")  # use this 